In [1]:
using JuMP, GLPK
using DataFrames
using CSV

In [2]:
##Creation of the Input DataFrames

#load demand data
Demands = CSV.File("../data Assign 1/Demands.csv") |> DataFrame

#load conventional generators data
Conv_gen = CSV.File("../data Assign 1/Conventional_generators.csv") |> DataFrame

#load wind farms data
Wind_farms = CSV.File("../data Assign 1/Wind_farms.csv") |> DataFrame

#load transmission lines data
Trans_lines = CSV.File("../data Assign 1/transmission_lines.csv") |> DataFrame

Row,Transmission lines: From node,To node,Susceptance [per-unit],Capacity [MW]
,Int64,Int64,String7,Int64
1,1,2,0.0146,175
2,1,3,0.2253,175
3,1,5,0.0907,350
4,2,4,0.1356,175
5,2,6,0.205,175
6,3,9,0.1271,175
7,3,24,0.084,400
8,4,9,0.111,175
9,5,10,0.094,350


In [3]:
##Definition of the Result DataFrames

# Definition of the Main result DataFrame
main_result = DataFrame(hour = Int[], social_welfare = Float64[], market_price = Float64[])

# Initialize the Demand DataFrame directly without dynamic column names
details_results_demand = DataFrame(hour = Int[], total_demand = Float64[])
# For x variables, manually add each column. This is a one-time setup.
for i in 1:size(Demands, 1)
    details_results_demand[!, Symbol("demand_n°$i")] = Float64[]
end

# Initialize the Supply DataFrame directly without dynamic column names
details_results_supply = DataFrame(hour = Int[], total_supply = Float64[])
# For x variables, manually add each column. This is a one-time setup.
for i in 1:size(Conv_gen, 1)
    details_results_supply[!, Symbol("supply_n°$i")] = Float64[]
end
for i in size(Conv_gen, 1):(size(Conv_gen, 1)+size(Wind_farms, 1))
    details_results_supply[!, Symbol("supply_n°$i")] = Float64[]
end


In [4]:
##Definition of the model

# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [16]:
##Computation of the optimization problem for every hour

#Clearing the DataFrames
empty!(main_result)
empty!(details_results_demand)
empty!(details_results_supply)


for hour in 1:24
    # Define the decision variables for every supply and every demand
    unregister(model, :x)  # Unregister the existing variable named "x" from the model
    unregister(model, :BalanceConstraint)
    @variable(model, x[1:size(Conv_gen, 1) + size(Wind_farms, 1) + size(Demands, 1)])  # Create the new variables

    for i in 1:size(Conv_gen, 1)
        set_lower_bound(x[i], 0)  # Set the lower bound
        set_upper_bound(x[i], Conv_gen[i,6])  # Set the upper bound
    end
    for i in (size(Conv_gen, 1) + 1):(size(Conv_gen, 1)+size(Wind_farms, 1))
        set_lower_bound(x[i], 0)  # Set the lower bound
        set_upper_bound(x[i], Wind_farms[i - size(Conv_gen, 1),4])  # Set the upper bound
    end
    for i in (size(Conv_gen, 1)+size(Wind_farms, 1) + 1):(size(Conv_gen, 1) + size(Wind_farms, 1) + size(Demands, 1))
        set_lower_bound(x[i], 0)  # Set the lower bound
        set_upper_bound(x[i], Demands[i - size(Conv_gen, 1) - size(Wind_farms, 1),3])  # Set the upper bound
    end

    # Define the objective function
    @objective(model, Min, 
    - (sum(Demands[demand,4] * x[demand + size(Conv_gen, 1) + size(Wind_farms, 1)] for demand in 1:size(Demands,1))
    - (sum(Conv_gen[generator,3] * x[generator] for generator in 1:size(Conv_gen,1))
    + sum( 0 * x[generator + size(Conv_gen,1)] for generator in 1:size(Wind_farms,1)))))

     # Add the equilibrium constraint
     @constraint(model, BalanceConstraint, sum(x[generator] for generator in 1:size(Conv_gen,1)+size(Wind_farms,1)) == sum(x[demand + size(Conv_gen, 1) + size(Wind_farms, 1)] for demand in 1:size(Demands,1)))

    # Print the model
    #print(model)

    # Solve the model
    optimize!(model)

    status = termination_status(model)
    if status == MOI.OPTIMAL
        println("Hour ", hour, ": Optimal solution found")

        # Assuming 'model' and 'x' are properly defined and 'hour' is known
        new_mainrow = [hour, -objective_value(model),dual(BalanceConstraint)]
        push!(main_result, new_mainrow)

        new_demandrow = [hour, sum(value(x[i + size(Conv_gen, 1)+size(Wind_farms, 1)]) for i in 1:size(Demands, 1))]
        append!(new_demandrow, [value(x[i + size(Conv_gen, 1)+size(Wind_farms, 1)]) for i in 1:size(Demands, 1)])
        push!(details_results_demand, new_demandrow)

        new_supplyrow = [hour, sum(value(x[i]) for i in 1:size(Conv_gen, 1)+size(Wind_farms, 1))]
        append!(new_supplyrow, [value(x[i]) for i in 1:size(Conv_gen, 1)+size(Wind_farms, 1)])
        push!(details_results_supply, new_supplyrow)


    elseif status == MOI.INFEASIBLE
        println("Hour ", hour, ": No feasible solution found")
    else
        println("Hour ", hour, ": Solver status: ", status)
    end
end

    

Hour 1: Optimal solution found


Hour 2: Optimal solution found
Hour 3: Optimal solution found
Hour 4: Optimal solution found
Hour 5: Optimal solution found
Hour 6: Optimal solution found
Hour 7: Optimal solution found
Hour 8: Optimal solution found
Hour 9: Optimal solution found
Hour 10: Optimal solution found
Hour 11: Optimal solution found
Hour 12: Optimal solution found
Hour 13: Optimal solution found
Hour 14: Optimal solution found
Hour 15: Optimal solution found
Hour 16: Optimal solution found
Hour 17: Optimal solution found
Hour 18: Optimal solution found
Hour 19: Optimal solution found
Hour 20: Optimal solution found
Hour 21: Optimal solution found
Hour 22: Optimal solution found
Hour 23: Optimal solution found
Hour 24: Optimal solution found


In [23]:
#save result_df to csv
CSV.write("../data Assign 1/Results/main_result_Max", main_result)
CSV.write("../data Assign 1/Results/details_results_demand_Max", details_results_demand)
CSV.write("../data Assign 1/Results/details_results_supply_Max", details_results_supply)


"../data Assign 1/Results/details_results_supply_Max"